# Record Merging
This *.ipynb file is used to merge records of the original dataset

In [ ]:
# importing packages
import ast
import pandas as pd
import os

# a helper function
def output(outname,df):
    outdir = './'
    if not os.path.exists(outdir):
        os.mkdir(outdir)

    fullname = os.path.join(outdir, outname)    
    df.to_csv(fullname)


First, we normalized the data by replacing the inconsistent Missing value representation of the data, for consistency, we changed them all into Missing. Then, we saved them as "1_Normalized_data.csv".

In [ ]:
data = pd.read_csv('Cleaned_data_v8.csv')
data.fillna("Missing",inplace = True)
data.replace("MISS","Missing",inplace= True)
data.replace(" MISS","Missing",inplace= True)
data.replace("--","Missing",inplace= True)
output("1_Normalized_data.csv",data)


Second, we filter out the records which have the same value in "SOURCE: Folder No MOD","APPLICATION: Year filed MOD","APPLICATION: Month MOD", and "APPLICATION: Day MOD" with another record, which means that these records have duplicates(i.e. filled out multiple time by Lab RAs). We stored the records we found in the file "3_Duplicated.csv".

We also filter out the records that do not have the same value as other records in "SOURCE: Folder No MOD","APPLICATION: Year filed MOD","APPLICATION: Month MOD", "APPLICATION: Day MOD", which means that these records have no duplicates(i.e. filled out only one time by Lab RAs), We stored the records we found in the file "2_UnDuplicated.csv"


In [ ]:
data = pd.read_csv('1_Normalized_data.csv')
#find dupicate by file and output
df1=data.groupby(["SOURCE: Folder No MOD","APPLICATION: Year filed MOD","APPLICATION: Month MOD",
"APPLICATION: Day MOD"]).size()
col=df1[df1==1].reset_index()[["SOURCE: Folder No MOD","APPLICATION: Year filed MOD","APPLICATION: Month MOD",
"APPLICATION: Day MOD"]]
a = pd.merge(col,data,on=["SOURCE: Folder No MOD","APPLICATION: Year filed MOD","APPLICATION: Month MOD",
"APPLICATION: Day MOD"])
a.drop(columns='Unnamed: 0',inplace=True)
output("2_UnDuplicated.csv",a)

data = pd.read_csv('1_Normalized_data.csv')
#find dupicate by file and output
df1=data.groupby(["SOURCE: Folder No MOD","APPLICATION: Year filed MOD","APPLICATION: Month MOD",
"APPLICATION: Day MOD"]).size()
col=df1[df1>1].reset_index()[["SOURCE: Folder No MOD","APPLICATION: Year filed MOD","APPLICATION: Month MOD",
"APPLICATION: Day MOD"]]
b = pd.merge(col,data,on=["SOURCE: Folder No MOD","APPLICATION: Year filed MOD","APPLICATION: Month MOD",
"APPLICATION: Day MOD"])
b.drop(columns='Unnamed: 0',inplace=True)
print(b)
output("3_Duplicated.csv",b)

Third, we merge the duplicated records in the file "3_Duplicated.csv".


We group the duplicated records when they have had the same value in "SOURCE: Folder No MOD","APPLICATION: Year filed MOD","APPLICATION: Month MOD", and "APPLICATION: Day MOD". When we grouped the duplicated records into one record, only the unique answers in each field is saved(the order of the answer is consistent with the index of the record, the smaller the index is, the answer will be present in front). For example. Duplicate A,B,C has the same value in "SOURCE: Folder No MOD","APPLICATION: Year filed MOD","APPLICATION: Month MOD", and "APPLICATION: Day MOD". Duplicate A have value of 1 in the field ExampleField, Duplicate B has value of 1 in the field ExampleField, Duplicate C has value of 2 in the field ExampleField. The final merged record will have value 1,2 in the field ExampleField. The merged record is saved in "5_Clean_Merge_Dupicated.csv".

In [ ]:

data = pd.read_csv('3_Duplicated.csv')
#merge data
df1 = data.astype(str).groupby(["SOURCE: Folder No MOD","APPLICATION: Year filed MOD","APPLICATION: Month MOD","APPLICATION: Day MOD"], as_index=False).agg(list)
df = df1.drop(columns=['Unnamed: 0'])
output("4_Merge_Duplicated.csv",df)


data = pd.read_csv('4_Merge_Duplicated.csv',dtype=str)
#clean merge data
def fx(i):
    if type(i) is str and '[' in i:
        x = ast.literal_eval(i)
        x = [n.strip() for n in x]
        print(type(x))
        result = list(dict.fromkeys(x))
        while result.count("Missing") >= 1 and len(result) >1:
            result.remove("Missing")
        fr = ','.join(result)
        return fr
    else:
        return i
mydata = data.applymap(fx)
df = mydata.drop(columns=['Unnamed: 0'])
output("5_Clean_Merge_Dupicated.csv",df)

Forth, we combined the "5_Clean_Merge_Dupicated.csv" file and "2_UnDuplicated.csv" file into a single file and then we reorganized the field, moving some important fields ('SOURCE: Folder Number','SOURCE: Folder No MOD',"APPLICATION: Year filed","APPLICATION: Year filed MOD", etc) in the front for better view. We stored it in the file "7_Cleaned_data_v9.csv".

In [ ]:

#combina two data set
data1 = pd.read_csv('5_Clean_Merge_Dupicated.csv')
data2 = pd.read_csv('2_UnDuplicated.csv')
frames = [data1,data2]
result = pd.concat(frames)
df = result.drop(columns=['Unnamed: 0'])
df_id = df["APPLICATION: Year filed"]
df = df.drop('APPLICATION: Year filed',axis=1)
df.insert(1,'APPLICATION: Year filed',df_id)
output("6_Cleaned_data_v9.csv",df)

# #reorganized data
data = pd.read_csv('6_Cleaned_data_v9.csv')
df = data.drop(columns=['Unnamed: 0'])

df_id = df["SOURCE: Folder Number"]
df = df.drop('SOURCE: Folder Number',axis=1)
df.insert(0,'SOURCE: Folder Number',df_id)

df_id = df["SOURCE: Folder No MOD"]
df = df.drop('SOURCE: Folder No MOD',axis=1)
df.insert(1,'SOURCE: Folder No MOD',df_id)

df_id = df["APPLICATION: Year filed"]
df = df.drop('APPLICATION: Year filed',axis=1)
df.insert(2,'APPLICATION: Year filed',df_id)

df_id = df["APPLICATION: Year filed MOD"]
df = df.drop('APPLICATION: Year filed MOD',axis=1)
df.insert(3,'APPLICATION: Year filed MOD',df_id)

df_id = df["APPLICATION: Month filed"]
df = df.drop('APPLICATION: Month filed',axis=1)
df.insert(4,'APPLICATION: Month filed',df_id)

df_id = df["APPLICATION: Month MOD"]
df = df.drop('APPLICATION: Month MOD',axis=1)
df.insert(5,'APPLICATION: Month MOD',df_id)

df_id = df["APPLICATION: Day filed"]
df = df.drop('APPLICATION: Day filed',axis=1)
df.insert(6,'APPLICATION: Day filed',df_id)

df_id = df["APPLICATION: Day MOD"]
df = df.drop('APPLICATION: Day MOD',axis=1)
df.insert(7,'APPLICATION: Day MOD',df_id)

df_id = df["#ID"]
df = df.drop('#ID',axis=1)
df.insert(0,'#ID',df_id)
# df['#ID'] = range(1, len(df) + 1)
# df_id = df["#ID"]
# df = df.drop('#ID',axis=1)
# df.insert(0,'#ID',df_id)


output("7_Cleaned_data_v9.csv",df)

Finally, we categorized the data into 3 types
- Family data
- Single data W/ multiple input
- Single data W/ 1 input


If the record has child/wife/Daugther in its field 'APPLICATION: Type', we marked the record as family data. Meaning that this record has merged the information of a family (Why? because family member have the same file name and application date, which meet the merging criteria we mentioned above)


If the record has no child/wife/Daugther in its field 'APPLICATION: Type', and it is a merged record, we marked the record as Single data W/ multiple input.


If the record has no child/wife/Daugther in its field 'APPLICATION: Type', and it is a not merged record, we marked the record as Single data W/ 1 input.


This information is stored in the field "DATA TYPE".


After we create the field "DATA TYPE" for every record, we give them a unique ID that is the mixture of ascii_uppercase char and number digits of length 6.


We stored the file in "DATA.csv".


In [ ]:

#Get coloums that contain family info, mulitpule SOURCE: Name on File or APPLICATION: Type MOD DETAILED
data = pd.read_csv('7_Cleaned_data_v9.csv')


family_data = data[data['APPLICATION: Type'].str.contains(',')]
family_data_Child = family_data[family_data['APPLICATION: Type'].str.contains('Child')]

family_data_Wife = family_data[family_data['APPLICATION: Type'].str.contains('Wife')]

family_data_child = family_data[family_data['APPLICATION: Type'].str.contains('child')]

family_data_wife = family_data[family_data['APPLICATION: Type'].str.contains('wife')]

family_data_d = family_data[family_data['APPLICATION: Type'].str.contains('Daugther')]


family_data1 = pd.concat([family_data_Child,family_data_Wife, family_data_child,family_data_wife,family_data_d]).drop_duplicates(keep= "first")

not_family_data = pd.concat([data, family_data1, family_data1]).drop_duplicates(keep=False)

multi_data1 = not_family_data[not_family_data['Timestamp'].str.contains(',')]

left_over = pd.concat([not_family_data, multi_data1, multi_data1]).drop_duplicates(keep=False)
print(len(data))
print(len(family_data1))
print(len(multi_data1))
print(len(left_over))

family_data1.drop(columns='Unnamed: 0',inplace=True)
multi_data1.drop(columns='Unnamed: 0',inplace=True)
left_over.drop(columns='Unnamed: 0',inplace=True)


#add type
family_data1['DATA TYPE'] = ["Family data"]*len(family_data1)
df_id = family_data1["DATA TYPE"]
family_data1 = family_data1.drop('DATA TYPE',axis=1)
family_data1.insert(0,'DATA TYPE',df_id)

multi_data1['DATA TYPE'] = ["Single data W/ multiple input"]*len(multi_data1)
df_id = multi_data1["DATA TYPE"]
multi_data1 = multi_data1.drop('DATA TYPE',axis=1)
multi_data1.insert(0,'DATA TYPE',df_id)

left_over['DATA TYPE'] = ["Single data W/ 1 input"]*len(left_over)
df_id = left_over["DATA TYPE"]
left_over = left_over.drop('DATA TYPE',axis=1)
left_over.insert(0,'DATA TYPE',df_id)


#combine
wholedata = pd.concat([family_data1,multi_data1, left_over])

#add unique id
import random
import string
import numpy as np
def id_generator(size=6, chars=string.ascii_uppercase + string.digits):
    return ''.join(random.SystemRandom().choice(chars) for _ in range(size))

wholedata['ID'] = wholedata['#ID'].apply(lambda x : id_generator(5))

df_id = wholedata["ID"]
wholedata = wholedata.drop('ID',axis=1)
wholedata.insert(0,'ID',df_id)



wholedata.reset_index(inplace=True,drop = True)


wholedata.replace('Missing', np.NaN,inplace= True)

output("DATA.csv",wholedata)